## Imports

In [2]:
import pandas as pd
import numpy as np
import datetime
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Dropout
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


## Récupération des data

In [3]:
btcusd_data = pd.read_csv('./data/btcusd_1-min_data.csv')

print(f"Dataset BTC/USD original shape: {btcusd_data.shape}")
print(f"Colonnes: {btcusd_data.columns.tolist()}")

# convertir le Timestamp Unix en datetime
btcusd_data['Timestamp'] = pd.to_datetime(btcusd_data['Timestamp'], unit='s')

# trier par timestamp
btcusd_data = btcusd_data.sort_values('Timestamp').reset_index(drop=True)

# 1 donnée par heure
btcusd_data.set_index('Timestamp', inplace=True)
btcusd_daily = btcusd_data['Close'].resample('H').last()
btcusd_daily = btcusd_daily.dropna().reset_index()
btcusd_daily.columns = ['Date', 'Close']

print(f"\nDataset BTC/USD réduit à 1 par heure : {btcusd_daily.shape}")
print(f"\nPremières lignes:")
print(btcusd_daily.head())
print(f"\nDernières lignes:")
print(btcusd_daily.tail())

# utiliser la colonne close
data = btcusd_daily['Close'].values.reshape(-1, 1)
print(f"\nDonnées préparées pour le modèle: shape = {data.shape}")

Dataset BTC/USD original shape: (7258717, 6)
Colonnes: ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']


C:\Users\33662\AppData\Local\Temp\ipykernel_16652\2214997775.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  btcusd_daily = btcusd_data['Close'].resample('H').last()



Dataset BTC/USD réduit à 1 par heure : (120980, 2)

Premières lignes:
                 Date  Close
0 2012-01-01 10:00:00   4.58
1 2012-01-01 11:00:00   4.58
2 2012-01-01 12:00:00   4.58
3 2012-01-01 13:00:00   4.58
4 2012-01-01 14:00:00   4.58

Dernières lignes:
                      Date     Close
120975 2025-10-20 19:00:00  110840.0
120976 2025-10-20 20:00:00  111133.0
120977 2025-10-20 21:00:00  110595.0
120978 2025-10-20 22:00:00  110860.0
120979 2025-10-20 23:00:00  110595.0

Données préparées pour le modèle: shape = (120980, 1)


## Callback

In [4]:
os.makedirs('logs', exist_ok=True)
os.makedirs('models', exist_ok=True)


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# si le modèle ne s'améliore pas pendant 5 epochs, arrêter l'entraînement
early_stopping = EarlyStopping(
    monitor='loss',
    patience=5,
    verbose=1,
    restore_best_weights=True
)

# si le modèle ne s'améliore pas pendant 3 epochs, réduire le taux d'apprentissage
reduce_lr = ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-7
)

# sauvegarder le meilleur modèle
model_checkpoint = ModelCheckpoint(
    'models/best_bitcoin_model.h5',
    monitor='loss',
    save_best_only=True,
    verbose=1
)

callbacks = [tensorboard_callback, early_stopping, reduce_lr, model_checkpoint]


## Normalisation, model, entraînement


In [5]:
# normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

look_back = 60  # utiliser 60 heures pour prédire le jour suivant
batch_size = 32

# créer le générateur de séquences
train_generator = TimeseriesGenerator(data_normalized, data_normalized,
                                      length=look_back, batch_size=batch_size)

print(f"Nombre de séquences d'entraînement: {len(train_generator)}")
print(f"Look back: {look_back} heures")
print(f"Batch size: {batch_size}")

# model LSTM
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(look_back, 1)),
    LSTM(100, return_sequences=True, input_shape=(look_back, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dense(25, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

print(f"\nRésumé du modèle:")
model.summary()

# training du model
print(f"\nEntraînement du modèle...")
print(f"TensorBoard logs: {log_dir}")
history = model.fit(
    train_generator,
    epochs=10,
    verbose=1,
    callbacks=callbacks
)

print(f"\nEntraînement terminé!")
print(f"Meilleur modèle sauvegardé dans: models/best_bitcoin_model.h5")

Nombre de séquences d'entraînement: 3779
Look back: 60 heures
Batch size: 32


c:\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Résumé du modèle:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 58, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 58, 100)        │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 58, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

3779/3779 ━━━━━━━━━━━━━━━━━━━━ 368s 97ms/step - loss: 2.4196e-04 - mae: 0.0090 - learning_rate: 0.0010
Epoch 5/10
   7/3779 ━━━━━━━━━━━━━━━━━━━━ 7:30 120ms/step - loss: 1.5898e-05 - mae: 0.0031Epoch 5/10
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 1.2895e-04 - mae: 0.0069
Epoch 5: loss improved from 0.00025 to 0.00017, saving model to models/best_bitcoin_model.h5

Epoch 5: loss improved from 0.00025 to 0.00017, saving model to models/best_bitcoin_model.h5


3779/3779 ━━━━━━━━━━━━━━━━━━━━ 369s 98ms/step - loss: 1.2896e-04 - mae: 0.0069 - learning_rate: 0.0010
Epoch 6/10
   1/3779 ━━━━━━━━━━━━━━━━━━━━ 13:47 219ms/step - loss: 9.8021e-05 - mae: 0.0093Epoch 6/10
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 1.5025e-04 - mae: 0.0071
Epoch 6: loss improved from 0.00017 to 0.00015, saving model to models/best_bitcoin_model.h5

Epoch 6: loss improved from 0.00017 to 0.00015, saving model to models/best_bitcoin_model.h5


3779/3779 ━━━━━━━━━━━━━━━━━━━━ 348s 92ms/step - loss: 1.5025e-04 - mae: 0.0071 - learning_rate: 0.0010
Epoch 7/10
   2/3779 ━━━━━━━━━━━━━━━━━━━━ 6:32 104ms/step - loss: 1.0424e-05 - mae: 0.0027 Epoch 7/10
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 2.1406e-04 - mae: 0.0084
Epoch 7: loss did not improve from 0.00015
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 329s 87ms/step - loss: 2.1405e-04 - mae: 0.0084 - learning_rate: 0.0010
Epoch 8/10

Epoch 7: loss did not improve from 0.00015
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 329s 87ms/step - loss: 2.1405e-04 - mae: 0.0084 - learning_rate: 0.0010
Epoch 8/10
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 1.3656e-04 - mae: 0.0061
Epoch 8: loss did not improve from 0.00015
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 370s 98ms/step - loss: 1.3656e-04 - mae: 0.0061 - learning_rate: 0.0010
Epoch 9/10

Epoch 8: loss did not improve from 0.00015
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 370s 98ms/step - loss: 1.3656e-04 - mae: 0.0061 - learning_rate: 0.0010
Epoch 9/10
3779/3779 ━

3779/3779 ━━━━━━━━━━━━━━━━━━━━ 374s 99ms/step - loss: 8.2536e-05 - mae: 0.0055 - learning_rate: 0.0010
Epoch 10/10
Epoch 10/10
3779/3779 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 3.5521e-05 - mae: 0.0037
Epoch 10: loss improved from 0.00010 to 0.00004, saving model to models/best_bitcoin_model.h5

Epoch 10: loss improved from 0.00010 to 0.00004, saving model to models/best_bitcoin_model.h5


3779/3779 ━━━━━━━━━━━━━━━━━━━━ 380s 100ms/step - loss: 3.5523e-05 - mae: 0.0037 - learning_rate: 5.0000e-04
Restoring model weights from the end of the best epoch: 10.

Entraînement terminé!
Meilleur modèle sauvegardé dans: models/best_bitcoin_model.h5
Restoring model weights from the end of the best epoch: 10.

Entraînement terminé!
Meilleur modèle sauvegardé dans: models/best_bitcoin_model.h5


## Évaluation et visualisations

In [ ]:
# créer le générateur de test
test_generator = TimeseriesGenerator(data_normalized, data_normalized,
                                     length=look_back, batch_size=1)

# faire les prédictions
predictions_normalized = model.predict(test_generator, verbose=0)

# inverser la normalisation
predictions = scaler.inverse_transform(predictions_normalized)

# récupérer les vraies valeurs
real_values = data[look_back:]

# créer les visualisations
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# graphique 1: Toutes les données quotidiennes
axes[0].plot(real_values, label='Vraies Valeurs', linewidth=2, marker='o', markersize=4, alpha=0.7)
axes[0].plot(predictions, label='Prédictions', linewidth=2, marker='s', markersize=4, alpha=0.7)
axes[0].set_title('Prédiction du cours Bitcoin (BTC/USD) - Données quotidiennes', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Jours')
axes[0].set_ylabel('Prix (USD)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Graphique 2: Zoom sur les 60 dernières heures
start_idx = max(0, len(real_values) - 60)
axes[1].plot(real_values[start_idx:], label='Vraies Valeurs', linewidth=2, marker='o', markersize=6)
axes[1].plot(predictions[start_idx:], label='Prédictions', linewidth=2, marker='s', markersize=6)
axes[1].set_title('Zoom sur les 60 dernières heures', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Heures')
axes[1].set_ylabel('Prix (USD)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculer les métriques d'erreur
mse = mean_squared_error(real_values, predictions)
mae = mean_absolute_error(real_values, predictions)
rmse = np.sqrt(mse)

print(f"\nMétriques de performance:")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"MAE (Mean Absolute Error): ${mae:.2f}")
print(f"RMSE (Root Mean Squared Error): ${rmse:.2f}")
print(f"Nombre de prédictions: {len(predictions)}")
print(f"\nPrix moyen prédit: ${predictions.mean():.2f}")
print(f"Prix moyen réel: ${real_values.mean():.2f}")